<a href="https://colab.research.google.com/github/kyxyxn/Telemedicine/blob/main/%5Bmental%5D_create_samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
import numpy as np
import pandas as pd   
from functools import reduce

In [41]:
rootPath ='/content/drive/MyDrive/ASU/CIS791_Chen_Final/uds-data/'
filenames_uds = [rootPath+'uds-2014-full-dataset.xlsx',
                 rootPath+'uds-2015-full-dataset.xlsx',
                 rootPath+'uds-2016-full-dataset.xlsx',
                 rootPath+'uds-2017-full-dataset.xlsx',
                 rootPath+'uds-2018-full-dataset.xlsx',
                 rootPath+'uds-2019-full-dataset.xlsx',
                 rootPath+'uds-2020-full-dataset.xlsx']

In [42]:
df = pd.read_csv(rootPath+'uds_mental_tele_df.csv')
# grant_list = list(set(df.grant_number.values.tolist()))

### Sample selection

- We want HCs that do not have Mobile Vans
- HCs that are not operating in foreign land
- HCs that operate within a single state (**OR COUNTY**)

In [64]:
def get_site_df(filenames):
  site_df = pd.DataFrame()
  for i in range(len(filenames)):
    print(i)
    temp_df = pd.read_excel(filenames[i], sheet_name='HealthCenterSiteInfo')
    temp_df['year']=filenames[i].split('uds-')[-1][:4]
    site_df = site_df.append(temp_df).reset_index(drop=True)
  return site_df
  
site_df = get_site_df(filenames_uds)
site_df = site_df.replace(to_replace={'-':np.nan})

# Fill in missing values
site_df.loc[site_df['Grant Number']=='H80CS00006', ['Site State']] = 'MA'
site_df.loc[site_df['Grant Number']=='H80CS00011', ['Site State']] = 'NY'
site_df.loc[site_df['Grant Number']=='H80CS00051', ['Site State']] = 'CA'
site_df.loc[site_df['Grant Number']=='H80CS00142', ['Site State']] = 'CA'
site_df.loc[site_df['Grant Number']=='H80CS00159', ['Site State']] = 'NY'
site_df.loc[site_df['Grant Number']=='H80CS00176', ['Site State']] = 'MD'
site_df.loc[site_df['Grant Number']=='H80CS00486', ['Site State']] = 'NC'
site_df.loc[site_df['Grant Number']=='H80CS00489', ['Site State']] = 'PR'
site_df.loc[site_df['Grant Number']=='H80CS00754', ['Site State']] = 'LA'
site_df.loc[site_df['Grant Number']=='H80CS01128', ['Site State']] = 'AK'
site_df.loc[site_df['Grant Number']=='H80CS01130', ['Site State']] = 'AK'
site_df.loc[site_df['Grant Number']=='H80CS02467', ['Site State']] = 'PW'
site_df.loc[site_df['Grant Number']=='H80CS04287', ['Site State']] = 'TX'
site_df.loc[site_df['Grant Number']=='H80CS06655', ['Site State']] = 'NH'
site_df.loc[site_df['Grant Number']=='H80CS08232', ['Site State']] = 'AK'
site_df.loc[site_df['Grant Number']=='H80CS11274', ['Site State']] = 'NY'

0
1
2
3
4
5
6


In [44]:
site_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78099 entries, 0 to 78098
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   BHCMIS ID                        78099 non-null  object
 1   Grant Number                     78099 non-null  object
 2   Health Center Name               78099 non-null  object
 3   Site Name                        78099 non-null  object
 4   Site Type                        78099 non-null  object
 5   Site Status                      78099 non-null  object
 6   Location Type                    78099 non-null  object
 7   Location Setting                 78099 non-null  object
 8   Operational Schedule             51862 non-null  object
 9   Calendar Schedule                52404 non-null  object
 10  Total Weekly Hours Of Operation  77576 non-null  object
 11  Service Area Population          78067 non-null  object
 12  Site Operated By                

In [65]:
drop_list = []
# Operates mobile vanes
drop_list += site_df[site_df['Location Type']=='Mobile Van']['Grant Number'].value_counts().index.tolist()
# Operates in multiple sites
temp_df = site_df[['Grant Number','Site State']].groupby(['Grant Number']).nunique()
drop_list += temp_df[temp_df['Site State']>1].index.values.tolist()
# Operates outside US state
drop_list += site_df[site_df['Site State'].isin(['MP', 'GU', 'AS', 'MH', 'VI', 'PW', 'FW', 'FM', 'AK', 'HI', 'PR'])]['Grant Number'].value_counts().index.tolist()

print("We should exclude {} HCs.".format(len(set(drop_list))))

site_df = site_df[~site_df['Grant Number'].isin(drop_list)].reset_index(drop=True)
print("We have {} HCs left for analysis.".format(len(site_df['Grant Number'].value_counts())))

We should exclude 545 HCs.
We have 892 HCs left for analysis.


#### Find zip codes --> County information

In [46]:
# Find zip codes --> County
missing_df = site_df[site_df['Site ZIP Code'].isna()].reset_index(drop=True)
non_missing_df = site_df[~site_df['Site ZIP Code'].isna()].reset_index(drop=True)
missing_df_dup = missing_df[['Grant Number', 'Site Street Address', 'Site ZIP Code', 'Site City', 'Site State']].drop_duplicates().reset_index(drop=True) # Only addresses
missing_df_dup['index'] = range(len(missing_df_dup))

# Merge with already found addresses & county codes
found_df = pd.read_csv('/content/drive/MyDrive/ASU/Telehealth/Data/healthcentersiteinfo.csv')
found_df = found_df[~found_df['zip_5'].isna()].reset_index(drop=True)
temp_df = pd.merge(missing_df_dup, found_df.rename(columns={'grant_number':'Grant Number'})[['Grant Number', 'Site Street Address', 'zip_5', 'county_fips']].drop_duplicates(), 
         on=['Grant Number', 'Site Street Address'], how='outer')
temp_df = temp_df[~temp_df['index'].isna()]
missing_df_dup = temp_df.copy()

temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1547 entries, 0 to 1546
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Grant Number         1547 non-null   object 
 1   Site Street Address  1535 non-null   object 
 2   Site ZIP Code        0 non-null      object 
 3   Site City            1547 non-null   object 
 4   Site State           1547 non-null   object 
 5   index                1547 non-null   float64
 6   zip_5                1436 non-null   float64
 7   county_fips          1396 non-null   float64
dtypes: float64(3), object(5)
memory usage: 108.8+ KB


### Select Health Centers

In [47]:
# Selected HCs from above:
grant_list = temp_df['Grant Number'].value_counts().index.tolist()

#### Health Centers

In [77]:
def healthcenterinfo(filenames):
  uds_df = pd.DataFrame()
  for i in range(len(filenames)):
    print(i)
    temp_df = pd.read_excel(filenames[i], sheet_name='HealthCenterInfo',
                            usecols=['Grant Number','BHCMIS ID', 'Reporting Year','Health Center Name', 'Urban Rural Flag'])
    temp_df = temp_df.rename(columns={'Grant Number':'grant_number', 'BHCMIS ID':'bhcmis_id', 'Reporting Year':'year', 'Urban Rural Flag':'rural_flag_center'})
    uds_df = uds_df.append(temp_df).reset_index(drop=True)
  return uds_df
uds_df = healthcenterinfo(filenames_uds)

# Selected HCs from above:
uds_df = uds_df[uds_df['grant_number'].isin(grant_list)].reset_index(drop=True)

0
1
2
3
4
5
6


#### Telehealth information

In [51]:
def teleinfo(filenames):
  temp_df = pd.DataFrame()
  for i in range(len(filenames_uds)):
    if i<2:
      continue
    elif i==2:
      uds_tele_df_2016 = pd.read_excel(filenames_uds[2], sheet_name='HITInformation')
      uds_tele_df_2016.columns = uds_tele_df_2016.iloc[0].values
      uds_tele_df_2016 = uds_tele_df_2016[['BHCMISID', 'GrantNumber', 'Tehr_L13_Ca', 'Tehr_L13a_Ca', 'Tehr_L13a_Cb']]
      uds_tele_df_2016['year'] = filenames[i].split('uds-')[-1][:4]
      uds_tele_df_2016.columns = ['BHCMIS ID', 'Grant Number', 'use_telehealth', 'how_telehealth', 'telehealth_other', 'year']
      
    elif i==3:
      uds_tele_df_2017 = pd.read_excel(filenames_uds[3], sheet_name='OtherDataElements')
      uds_tele_df_2017.columns = uds_tele_df_2017.iloc[0].values
      uds_tele_df_2017 = uds_tele_df_2017[['BHCMISID', 'GrantNumber', 'Tode_L2_Ca', 'Tode_L2a_Ca', 'Tode_L2a_Cb']]
      uds_tele_df_2017['year'] = filenames[i].split('uds-')[-1][:4]
      uds_tele_df_2017.columns = ['BHCMIS ID', 'Grant Number', 'use_telehealth', 'how_telehealth', 'telehealth_other', 'year']
      
    elif i==4:
      uds_tele_df_2018 = pd.read_excel(filenames_uds[4], sheet_name='OtherDataElements')
      uds_tele_df_2018.columns = uds_tele_df_2018.iloc[0].values
      uds_tele_df_2018 = uds_tele_df_2018[['BHCMISID', 'GrantNumber', 'Tode_L2_Ca','Tode_L2a3_Ca','Tode_L2a3_Other']]
      uds_tele_df_2018['year'] = filenames[i].split('uds-')[-1][:4]
      uds_tele_df_2018.columns = ['BHCMIS ID', 'Grant Number', 'use_telehealth', 'how_telehealth', 'telehealth_other', 'year']
      
    elif i==5:
      uds_tele_df_2019 = pd.read_excel(filenames_uds[5], sheet_name='OtherDataElements')
      uds_tele_df_2019.columns = uds_tele_df_2019.iloc[0].values
      uds_tele_df_2019 = uds_tele_df_2019[['BHCMISID', 'GrantNumber', 'Tode_L2_Ca','Tode_L2a3_Ca','Tode_L2a3_Other']]
      uds_tele_df_2019['year'] = filenames[i].split('uds-')[-1][:4]
      uds_tele_df_2019.columns = ['BHCMIS ID', 'Grant Number', 'use_telehealth', 'how_telehealth', 'telehealth_other', 'year']
      
    elif i==6:
      uds_tele_df_2020 = pd.read_excel(filenames_uds[6], sheet_name='OtherDataElements')
      uds_tele_df_2020.columns = uds_tele_df_2020.iloc[0].values
      uds_tele_df_2020 = uds_tele_df_2020[['BHCMISID', 'GrantNumber', 'Tode_L2_Ca','Tode_L2a3_Ca','Tode_L2a3_Other']]
      uds_tele_df_2020['year'] = filenames[i].split('uds-')[-1][:4]
      uds_tele_df_2020.columns = ['BHCMIS ID', 'Grant Number', 'use_telehealth', 'how_telehealth', 'telehealth_other', 'year']
   
  for i in [uds_tele_df_2016, uds_tele_df_2017, uds_tele_df_2018, uds_tele_df_2019, uds_tele_df_2020]:
    temp_df = temp_df.append(i)
  
  temp_df = temp_df[temp_df['Grant Number']!='GrantNumber'].reset_index(drop=True)
  return temp_df

uds_tele_df = teleinfo(filenames_uds)

In [53]:
# Clean
uds_tele_df['use_telehealth'] = uds_tele_df['use_telehealth'].replace({'Yes':1,'No':0})
uds_tele_df['how_telehealth'] = [str(i).lower() for i in uds_tele_df['how_telehealth']]
uds_tele_df['telehealth_other'] = [str(i).lower() for i in uds_tele_df['telehealth_other']]

# See if HC provides teleheealth for mental health services
matchers = ['mental', 'behav', 'substance', 'counseling', 'psych']

new_list = []
for s in uds_tele_df['how_telehealth']:
  if any(xs in s for xs in matchers):
    new_list.append(1)
  else:
    new_list.append(0)
uds_tele_df['how_telehealth_mental'] = new_list
    
new_list = []
for s in uds_tele_df['telehealth_other']:
  if any(xs in s for xs in matchers):
    new_list.append(1)
  else:
    new_list.append(0)
uds_tele_df['telehealth_other_mental'] = new_list

# Mental health (at least 1)
uds_tele_df['mental_telehealth'] = uds_tele_df['how_telehealth_mental'] + uds_tele_df['telehealth_other_mental']
uds_tele_df['mental_telehealth'] = [1 if i>=1 else 0 for i in uds_tele_df['mental_telehealth']]

uds_tele_df = uds_tele_df[['BHCMIS ID', 'Grant Number', 'use_telehealth', 'mental_telehealth', 'year']]

In [54]:
# long to wide
uds_tele_df_wide = uds_tele_df[['Grant Number', 'mental_telehealth', 'year']].pivot(index='Grant Number', columns='year', values='mental_telehealth')
uds_tele_df_wide.columns = ['mental_2016', 'mental_2017', 'mental_2018', 'mental_2019', 'mental_2020']

In [56]:
# inconsistent
drop_list1 = uds_tele_df_wide[(uds_tele_df_wide['mental_2016']==1)&((uds_tele_df_wide['mental_2017']==0)|(uds_tele_df_wide['mental_2018']==0)|(uds_tele_df_wide['mental_2019']==0)|(uds_tele_df_wide['mental_2020']==0))].index.tolist()
drop_list2 = uds_tele_df_wide[(uds_tele_df_wide['mental_2017']==1)&((uds_tele_df_wide['mental_2018']==0)|(uds_tele_df_wide['mental_2019']==0)|(uds_tele_df_wide['mental_2020']==0))].index.tolist()
drop_list3 = uds_tele_df_wide[(uds_tele_df_wide['mental_2018']==1)&((uds_tele_df_wide['mental_2019']==0)|(uds_tele_df_wide['mental_2020']==0))].index.tolist()
drop_list4 = uds_tele_df_wide[(uds_tele_df_wide['mental_2019']==1)&(uds_tele_df_wide['mental_2020']==0)].index.tolist()
drop_list = sorted(list(set(drop_list1+drop_list2+drop_list3+drop_list4)))
print("There are {} HCs that are inconsistent with their telehealth policies.".format(len(drop_list)))

uds_tele_df['consistency_1'] = [0 if i in drop_list else 1 for i in uds_tele_df['Grant Number']]

There are 211 HCs that are inconsistent with their telehealth policies.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [59]:
# inconsistent
drop_list1 = uds_tele_df_wide[(uds_tele_df_wide['mental_2016']==1)&((uds_tele_df_wide['mental_2017']==0)|(uds_tele_df_wide['mental_2018']==0)|(uds_tele_df_wide['mental_2019']==0))].index.tolist()
drop_list2 = uds_tele_df_wide[(uds_tele_df_wide['mental_2017']==1)&((uds_tele_df_wide['mental_2018']==0)|(uds_tele_df_wide['mental_2019']==0))].index.tolist()
drop_list3 = uds_tele_df_wide[(uds_tele_df_wide['mental_2018']==1)&((uds_tele_df_wide['mental_2019']==0))].index.tolist()
drop_list = sorted(list(set(drop_list1+drop_list2+drop_list3)))
print("There are {} HCs that are inconsistent with their telehealth policies.".format(len(drop_list)))

uds_tele_df['consistency_2'] = [0 if i in drop_list else 1 for i in uds_tele_df['Grant Number']]

There are 207 HCs that are inconsistent with their telehealth policies.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [74]:
uds_tele_df = uds_tele_df[uds_tele_df['Grant Number'].isin(grant_list)].reset_index(drop=True)

In [78]:
uds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3618 entries, 0 to 3617
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bhcmis_id           3618 non-null   object
 1   grant_number        3618 non-null   object
 2   year                3618 non-null   int64 
 3   Health Center Name  3618 non-null   object
 4   rural_flag_center   3618 non-null   object
dtypes: int64(1), object(4)
memory usage: 141.5+ KB


In [76]:
uds_tele_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2612 entries, 0 to 2611
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHCMIS ID          2612 non-null   object
 1   Grant Number       2612 non-null   object
 2   use_telehealth     2612 non-null   int64 
 3   mental_telehealth  2612 non-null   int64 
 4   year               2612 non-null   int64 
 5   consistency_1      2612 non-null   int64 
 6   consistency_2      2612 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 143.0+ KB


In [79]:
# Merge HC and telehealth data
uds_df.loc[:, 'year'] = uds_df.loc[:, 'year'].astype(int)
uds_tele_df.loc[:, 'year'] = uds_tele_df.loc[:, 'year'].astype(int)
temp_df = pd.merge(uds_df.rename(columns={'grant_number':'Grant Number'}), uds_tele_df.drop(columns=['BHCMIS ID']), on=['Grant Number', 'year'], how='outer')
temp_df = temp_df[~temp_df['bhcmis_id'].isna()].reset_index(drop=True)
temp_df = temp_df[(temp_df['year']<2016)|(~temp_df['use_telehealth'].isna())].reset_index(drop=True)
uds_df = temp_df.copy()

# Merge with Site data
site_df.loc[:, 'year'] = site_df.loc[:, 'year'].astype(int)
temp_df = pd.merge(uds_df, site_df[['Grant Number', 'year', 'Site State']].drop_duplicates(), on=['Grant Number', 'year'], how='outer')

temp_df.loc[temp_df['Grant Number']=='H80CS28999', 'Site State'] = 'CT'
temp_df.loc[temp_df['Grant Number']=='H80CS29013', 'Site State'] = 'NJ'
temp_df.loc[temp_df['Grant Number']=='H80CS28445', 'Site State'] = 'NY'
temp_df.loc[temp_df['Grant Number']=='H80CS29015', 'Site State'] = 'NJ'
temp_df.loc[temp_df['Grant Number']=='H80CS28977', 'Site State'] = 'TX'
temp_df.loc[temp_df['Grant Number']=='H80CS29048', 'Site State'] = 'CA'
temp_df.loc[temp_df['Grant Number']=='H80CS00566', 'Site State'] = 'MT'
temp_df.loc[temp_df['Grant Number']=='H80CS33655', 'Site State'] = 'NJ'
temp_df.loc[temp_df['Grant Number']=='H80CS33651', 'Site State'] = 'IN'

uds_df = temp_df.copy()

### Save

In [85]:
uds_df.to_csv(rootPath+'samples_hc.csv', index=False)